<p><img alt="Colaboratory logo" height="65px" src="https://upload.wikimedia.org/wikipedia/en/thumb/b/b1/Davivienda_logo.svg/1200px-Davivienda_logo.svg.png" align="left" hspace="10px" width="20%" vspace="15px"></p>

<h1 align="center"> Prueba Técnica Profesional III Departamento de Datos no Estructurados  </h1>


#### **Juan Sebastián Gómez Duque**
#### **Estadístico | Científico de datos - Facultad de Ciencias, Universidad Nacional de Colombia**
#### **Correo electrónico: jgomezd@unal.edu.co**



---



## Descripción del proceso

Un vistazo al proceso permite identificar que pueden ser muy útiles algunos métodos modernos como en los que se usan los transformadores de HugginFace. El proceso hace uso de dos de estos transformadores que se encuentran entrenados en español, dada la naturaleza del conjunto de datos y con el objetivo de extraer los tópicos principales del conjunto:
1. Se realiza una limpieza sobre el conjunto de tweets, uniendo emojis con los tweets, eliminando dígitos y quitando carácteres especiales (como @)
2. Se emplea un Embedding pre-entrenado de HugginFace conocido como "sentence_similarity_spanish_es" (esto convierte en un vector numérico cada uno de los tweets)
3. Para el uso del modelo BERTopics (el cual es nuestra principal motivación) requiere del uso de dos herramientas más: UMAP Y HBDSCAN, los cuales reducen la dimensión de los datos y generan clusters de datos, respectivamente. El proceso de extracción de tópicos en BERTopics genera un conjunto de palabras que se asocian en mayor medida a dicho tópico.
4. Del proceso de Cluster que se realizó por BERTopics, con ayuda de HBDSCAN, se obtienen los documentos más relevantes para cada tópico (Cluster) para así disminuir la influencia de documentos que no sean tan relevantes para generar una idea general de cada uno de los tópicos.
5. Del proceso de BERTopics se extraen las palabras más relacionadas con cada tópico y de estas se eliminan las stopwords para posteriormente presentarlas.
6. Por medio de un summary model llamado "bert2bert" con base en español se generan resumenes de cada uno de los tópicos. Para este proceso se genera un único texto para cada tópico, uniendo los tweets que más lo representan. Este proceso genera un nuevo producto que en compañia de los datos que se obtuvieron en el paso anterior, ayuda a visualizar con mayor claridad el concepto general de cada uno de los tópicos.


A continuación se visualiza un diagrama que representa el proceso que se está realizando en el modelaje de tópicos:

<img src="https://github.com/dux135/Prueba-Davivienda/blob/main/Texto/Diagrama_proceso.jpg?raw=true"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />

## 1. Importación y lectura de datos

**Importe de librerías**

Se realiza el proceso de instalación de librerías e importación de las mismas para poder correr las diferentes herramientas utilizadas en el proyecto

In [ ]:
!pip intsall nltk #La librería nltk se utilizará principalmente para el uso de diccionarios stopwords con el objetivo de
                  #depurar datos

In [ ]:
!pip install transformers #La librería transformers es útil para importar todos los modelos y herrmiaentas necesarias
                          #que se usarán de la plataforma HugginFace

In [ ]:
!pip install sentence_transformers #Librería necesaria pora el mannejo de transformadores, en particular se usará para
                                   #realizar el embedding inicial de los tweets

In [ ]:
!pip install umap-learn #Se instala la librería UMAP
import umap #La librería UMAP brinda una metodología útil para la representación de los datos y reducir la dimensionalidad 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install hdbscan #Se instala la librería hdbscan
import hdbscan #La librería hbdscan permite la creación de clusters y con esto se generan la diferencia entre documentos
               #por tópicos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Para el proceso de modelado de tópicos BERTopics se utiliza CUDA de Nvidia por lo tanto es necesario la instalación de los controladores y la versión adecuada de Torch

In [ ]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [ ]:
!pip install bertopic #Se instala BERTopics para el modelado de tópicos


In [ ]:
import pandas as pd #Se importa la librería pandas para el manejo de conjuntos de datos
import numpy as np #Se importa numpy para el manejo de datos y operaciones matemáticas

In [ ]:
import os #Se importa OS, librería útil para el manejo de ubicaciones y extraer los datos de las carpetas respectivas de
          #Google Drive

In [ ]:
import copy #Se importa copy, librería relevante para la generación de copias en conjuntos de datos, de esta manera
            #se conserva la integridad de los datos originales

In [ ]:
import nltk #Se importa nltk, librería útil para el preprocesamiento de lenguaje natural
nltk.download('stopwords') #Se descarga un conjunto de stopwords estandar de nltk
from nltk.corpus import stopwords #Se importa la función stopwords que se utilizará para obtener la lista de stopwords
                                  #en español que necesita para el manejo de tweets


In [ ]:
import torch #Es importante importar la librería torch, de la cual hará uso más adelante BERTopics 
from transformers import BertTokenizerFast, EncoderDecoderModel #Funciones utilizadas posteriormente para la tokenización e
                                                                #implementación del modelo summary bert2bert

In [ ]:
from bertopic import BERTopic #Se importa la librería BERTopics para el modelado de tópicos

In [ ]:
from sentence_transformers import SentenceTransformer #Se importa librería para realizar el embedding sobre los datos

In [ ]:
import re #Librería útil pora el manejo de expresiones regulares

**Importación de archivos**

Se establece la carpeta raíz donde se encuentra el conjunto de datos de tweets

In [ ]:
os.chdir("/content/drive/MyDrive/Davivienda/Prueba_conocimientos_davivienda/Ejercicio_2_Chats/Datasets")

In [ ]:
tweets=pd.read_csv("davivienda_tweets.csv") #Se importa el conjunto de datos en un DataFrame de pandas

## 2. Prepocesamiento de Texto

Se definen algunas funciones que serán de mucha utilidad para el proceso de procesamiento del texto

Por medio de la función stopwords de NLTK fue posible obtener un diccionario de stopwords, que será de mucha utilidad adelante para poder eliminar paalabras que no son el foco central del texto sino más bien herramientas del lenguaje común en español

In [ ]:
palabrasVacias = set(stopwords.words('spanish'))
palabrasVacias.add("davivienda") #Se agrega "davivienda" a la lista de stopwords, esto debido a que es nuestro tema principal entonces queda implicito
                                 #dentro del mensaje
print(palabrasVacias) #Se presenta el listado de palabras utilizado para el desarrollo del ejercicio

{'nuestros', 'soy', 'estabais', 'estar', 'esto', 'hayan', 'estando', 'quien', 'estaría', 'tendrán', 'estés', 'has', 'era', 'algo', 'ti', 'estaréis', 'tenía', 'hubiesen', 'estuviese', 'seamos', 'tendré', 'ni', 'seríais', 'davivienda', 'somos', 'habrá', 'fuese', 'estarían', 'estadas', 'también', 'del', 'han', 'estas', 'mi', 'cual', 'e', 'otros', 'ella', 'otro', 'sentido', 'habíamos', 'estaré', 'yo', 'habiendo', 'tenga', 'ante', 'vuestro', 'tanto', 'tuve', 'tengáis', 'estado', 'ese', 'hubieseis', 'de', 'hubieses', 'tuvimos', 'durante', 'tuviese', 'estuvo', 'fuimos', 'que', 'estuviéramos', 'tenías', 'habéis', 'vosotros', 'nosotros', 'donde', 'tuvieron', 'tengan', 'tuviera', 'este', 'desde', 'lo', 'fueras', 'estuvieron', 'estuviésemos', 'pero', 'fue', 'habidos', 'hubieras', 'estaremos', 'vuestra', 'tendremos', 'hubieron', 'como', 'sería', 'eras', 'fueseis', 'sentidas', 'fuerais', 'tuyos', 'suya', 'cuando', 'hasta', 'serán', 'seríamos', 'tus', 'sois', 'estos', 'hubiese', 'vuestras', 'habrías

Se define la función que en breve utilizaremos para eliminar las stopwords de una lista, llamada **quitar_stopwords** la cual toma las palabras definidas para cada tópico y elimina palabras que no pooseen relevancia dentro del contexto textual

In [ ]:
def quitar_stopwords(lista):
  aux_lista=[]
  for label in lista:
    if label not in palabrasVacias:
      aux_lista.append(label)
  return aux_lista

Se genera la función **remover_palabras_cortas** que elimina de una lista de palabras aquellas palabras con menos de 3 caracteres

In [ ]:
def remover_palabras_cortas(tweet_token):
  tokens_aux=[]
  for tok in tweet_token:
    if len(tok[0]) > 3:
        tokens_aux.append(tok)
  return tokens_aux

Se define la función **limpiar_tweets** que realiza algunos procesos sobre los tweets:
* Concatena texto del tweet con su emoji (si es que éste existe).
* Elimina caracteres especiales, dígitos (incluyendo el @ tan común en redes sociales) y saltos de linea.
* Deja en minuscula todo el texto

Esta función recibe un DataFrame en pandas y devuelve este mismo después de realizar todasd las modificaciones.

In [ ]:
def limpiar_tweets(DataFrame):
  aux_df=copy.copy(DataFrame)
  aux_df.loc[~aux_df["Emojis"].isna(),"Embedded_text"]=aux_df.loc[~aux_df["Emojis"].isna(),"Embedded_text"]+" "+aux_df.loc[~aux_df["Emojis"].isna(),"Emojis"]
  aux_df["Embedded_text"]=aux_df["Embedded_text"].apply(lambda x:re.sub(r'[()]','',re.sub(r'\d+', '',x)).replace("\n"," ").replace("@",""))
  aux_df["Embedded_text"]=aux_df["Embedded_text"].apply(lambda x:x.lower())  
  return aux_df

In [ ]:
tweets=limpiar_tweets(tweets)

## 3. Embedding y aplicación del modelo BERTopics

**Proceso de Embedding**

In [ ]:
# Se carga el modelo de transformación de sentencias con una base previamente entrenada en español (Que corresponde al idioma de los tweets)
sentence_model = SentenceTransformer("hiiamsid/sentence_similarity_spanish_es")

# Realiza el proceso de embedding para cada uno de los tweets
embeddings = sentence_model.encode(tweets["Embedded_text"], show_progress_bar=False)

**Generación del modelo**

In [ ]:
# Se define el modelo UMAP  para reducir la dimensionalidad de los datos luego del embedding, éste utiliza un número de vecinos cercanos de 15,
#10 componentes y utiliza como métrica a coseno
umap_model = umap.UMAP(n_neighbors=15,
                       n_components=10,
                       min_dist=0.0,
                       metric='cosine',
                       low_memory=False)

In [ ]:
# Se define el modelo HBDSCAN para generar los clisters, se definen además los parametros a utilizar como un mínimo de 10 tweets para cada cluster,
#al menos una muestra paras obtener los grupos, se utiliza la metrica euclidiana para clasificar en cada cluster, como método de selección de cluster
# se utiliza 'eom' o  Excess of Mass algorithm  (Algoritmo de exceso de masa)
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=10,
                                min_samples=1,
                                metric='euclidean',
                                cluster_selection_method='eom',
                                prediction_data=True)

In [ ]:
# Con base en los métodos de cluster y reducción de la dimensionalidad (HBDSCAN y UMAP respectivamente) se genera un primer modelo de BERTopics, 
# el cual brinda un conjunto de expresiones para la definición de cada tópico
topic_model = BERTopic(top_n_words=20,
                       n_gram_range=(1,2), 
                       calculate_probabilities=True,
                       umap_model= umap_model,
                       hdbscan_model=hdbscan_model,
                       verbose=True)

# Se realiza el entrenamiento y de este se extraen los tópicos y un vector de probabilidades de pertenencia a los diferentes tópicos  para cada tweet.
topics, probabilities = topic_model.fit_transform(tweets["Embedded_text"], embeddings)

2022-08-09 05:54:32,084 - BERTopic - Reduced dimensionality
2022-08-09 05:54:32,522 - BERTopic - Clustered reduced embeddings


## 4. Prepocesamiento posterior

**Documentos relevantes por tópico**

In [ ]:
def get_most_relevant_documents(cluster_id, condensed_tree):
          
    assert cluster_id > -1, "La categoria del tweet debe ser mayor a -1!"
        
    raw_tree = condensed_tree._raw_tree
    
    # Se excluyen puntos únicos y sólo se tienen en cuenta los elementos del arbol
    cluster_tree = raw_tree[raw_tree['child_size'] > 1]
    
    # Se obtienen los nodos para cada rama del arbol que se está considerando
    leaves = hdbscan.plots._recurse_leaf_dfs(cluster_tree, cluster_id)
    
    # Se toman los puntos maás relevantes de cada rama en el arbol
    result = np.array([])
    
    for leaf in leaves:
        max_lambda = raw_tree['lambda_val'][raw_tree['parent'] == leaf].max()
        points = raw_tree['child'][(raw_tree['parent'] == leaf) & (raw_tree['lambda_val'] == max_lambda)]
        result = np.hstack((result, points))
        
    return result.astype(np.int)

In [ ]:
# Se obtiene el modelo de cluster, el arbol de clusters y los ID's de tema para cada cluster.
clusterer = topic_model.hdbscan_model
tree = clusterer.condensed_tree_
clusters = tree._select_clusters()


In [ ]:
#Se genera una lista de los tweets más relevantes para cada uno de los tópicos generados
lista_elementos=pd.concat([pd.DataFrame(get_most_relevant_documents(clusters[i], tree)) for i in tweets_2.Topicos.unique()]).drop_duplicates()[0].to_list()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Se hace uso de esta lista para filtrar unicamente los tweets más relevantes dentro del conjunto de datos inicial

In [ ]:
# Se genera un DataFrame con la categorización de cada tweet dentro de un tópcio
tweets_2=pd.concat([tweets,pd.DataFrame(topics).rename(columns={0:"Topicos"})],axis=1)

In [ ]:
# Con este último DataFrame  tenemos la certeza de solo poseer los tweets más relevantes para cada tópico
tweets_2=tweets_2.iloc[lista_elementos]

## 5. Interpretación de Resultados y conclusiones generales.

**Expresiones relevantes por tópico**

Se genera un primer vistazo a los tópicos generados por el modelo BERTopcis y se aplica sobre el conjunto de datos las funciones **quitar_stopwords** y **remover_palabras_cortas**, lo cual nos asegura ver una lista de tópicos más limpia para cada tópico.

Más adelante se procede a detallar y realizar un análisis de cada uno de los tópicos (con los resultados de este ejercicio y el que posteriormente se obtendrá con otras metodologías)

In [ ]:

for i in topic_model.topics:
  print(f"Tópico {i}",quitar_stopwords(remover_palabras_cortas(topic_model.topics[i])))

Tópico -1 [('hola', 0.03808600200264107), ('respuesta', 0.028080388732386427), ('para', 0.028036878823205113), ('en respuesta', 0.026623189751053003), ('lo sucedido', 0.026389393864938035), ('sucedido', 0.025980512765610125), ('por favor', 0.02462442362867021), ('favor', 0.02441723195137361), ('mensaje', 0.02284908601983506), ('atencin', 0.02241858415940719), ('por mensaje', 0.020979216368972115), ('respondiendo', 0.019945743087047434)]
Tópico 0 [('davivienda', 0.03597118183250387), ('respuesta', 0.021158645019804568), ('en respuesta', 0.020340724182409488), ('para', 0.011725035896674204)]
Tópico 1 [('gusto lo', 0.07495216588701918), ('con gusto', 0.07350142615869872), ('gusto', 0.07332892834065875), ('privado con', 0.06867022108797423), ('lo validaremos', 0.06849087188197159), ('validaremos', 0.06849087188197159), ('validaremos quedamos', 0.06849087188197159), ('favor escrbanos', 0.06834040166353986), ('escrbanos', 0.06834040166353986), ('escrbanos por', 0.06834040166353986), ('lament

**Resumenes de ejemplo por tópico**

Todo el proceso construido hasta ahora es útil para la identificación subjetiva de tópicos en el conjunto de datos que contiene los tweets, sin embargo se busca ir más allá y generar información mucho más accesible para el usuario final. Para este caso se busca generar un resumen por tópico según se hayan generado la clasificación de tweet según su tematica.

Bajo esta idea, se hace uso del transformador tipo Encoder-Decoder'mrm8488/bert2bert_shared-spanish-finetuned-summarization' de la plataforma HugginFace que permite realizar resumenes de textos y nos brinda una idea amplia del tipo de información con la cual nos podremos topar.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Esta linea hace referencia a la activación de CUDA de Nvidia para poder realizar el proceso 
                                                        # que requiere el uso de la GPU de Colab
ckpt = 'mrm8488/bert2bert_shared-spanish-finetuned-summarization' # Se nombra el nombre del transformer a utilizar, esta elección se realizó dado
                                                                  # que es uno de los transformadores bert2bert mejor puntuados en HugginFace
tokenizer = BertTokenizerFast.from_pretrained(ckpt) #Se carga el tokenizador pre-entrenado con el modelo previamente definido
model_3 = EncoderDecoderModel.from_pretrained(ckpt).to(device) #Se carga el modelo de resumen que utilizaremos

#Se genera la función que regala un breve resumen  para un texto (que en este caso se usará la concatenación de tweets por tópico)

def generate_summary(text):

   inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = model_3.generate(input_ids, attention_mask=attention_mask)
   return tokenizer.decode(output[0], skip_special_tokens=True)
   

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [ ]:
# Se genera una función que recibe un DataFrame y devuelve un diccionario para el tópico que se le solicite
def resumen_topico(DataFrame,campo_texto,campo_topico,topico_nom):
  df_aux=copy.copy(DataFrame)
  df_aux=df_aux.query(f"{campo_topico}=={topico_nom}")
  combined=""
  for i in df_aux[campo_texto]:
    combined+=f"{i}. "
  return {'topico':topico_nom,'texto':generate_summary(combined)}

In [ ]:
resmune_topicos=[resumen_topico(tweets_2,'Embedded_text','Topicos',topico) for topico in tweets_2["Topicos"].unique()] #Se genera el proceso resumir cada 
                                                                                                                       #tópico por sus tweets más relevantes

In [ ]:
resmune_topicos

[{'texto': 'El concierto davivienda y la wellagency de sfcsupervisor en el décimo aniversario de la celebración navideña',
  'topico': 0},
 {'texto': 'En respuesta a las molestias ocasionadas, ofrecemos una selección de artículos de EL PAÍS',
  'topico': 10},
 {'texto': 'En respuesta a alfredmaggiore para revisar su caso por favor envíenos su nombre y número de documento, únicamente por mensaje privado',
  'topico': 8},
 {'texto': 'En respuesta a las urnas sra. erikasu, sra erikauu / buenas tardes, sr. camilo',
  'topico': 6},
 {'texto': 'En respuesta a la pregunta de lsssshhhh buenas tardes. lamentamos los inconvenientes presentados.',
  'topico': 1},
 {'texto': 'En respuesta a marianiniecheve hola maria, gracias por escribirnos, nos interesa conocer los detalles de su caso y confírmenos sus datos por mensaje interno',
  'topico': 7},
 {'texto': 'Ha sido atendida por mensaje interno y nos encontramos atentos para ayudar en caso de alguna inquietud adicional',
  'topico': 5},
 {'texto'

**Interpretación**

El tema de interpretación textual se puede presentar para ser algo muy subjetivo, por lo tanto se compararán ambos resultados que se obtuvieron en este proceso: las expresiones por tópico y el ejemplo de resumen por tópico. Cabe aclarar que el resumen se hace sobre todos los tweets de un mismo tópicos por lo tanto algunos de los resumenes pueden no tener sentido lógico pero pueden llegar a dar una idea de la intención de los textos. Aquí presento mi análisis subjetivos de cada uno de los tópicos.

* **Tópico 0**: Se relaciona principalmente una respuesta brindada por la compañia davivienda, que se podría interpretar sobre cualquier tema (eso incluyendo la de cualquier evento que pueda tener cabida la participación de la compañia como podría sugerir el resumen)

* **Tópico 1**: Este está más referido a la respuesta para una validación u inconvenientes presentados, como podemos inferir de las expresiones de la primera parte (es claro que el resumen podría ser un ejemplo claro de éste tema, como bien dice está relacionado a conocer más a profundidad y validar un caso "...buenas tardes. lamentamos los inconvenientes presentados.")

* **Tópico 2**: Va claramente relacionado con la solicitud de detalles de alguna problematica presentada por el usuario, aquí cabe resaltar que se suguiere la comunicación directa del cliente y además se brindan palabras que tranquilicen al usuario, además de mostrar la disposición de la compañia a buscar la mejora constante (el resumen está claramente relacionado con las expresiones generadas por BERTopics "Trabajar continuamente para mejorar su experiencia. Le pedimos detallar su caso por mensaje privado")

* **Tópico 3**: Aquí claramente se involucra el tema de molestias, se solicita el apoyo al usuario para diligenciar el caso y se brindan disculpas por percances en el camino (mucho mejor explicado por el ejemplo del resumen "Por favor escribanos por mensaje privado para validar su caso... ")

* **Tópico 4**: Se puede brindar como una respuesta a cualquier caso y se brindan los canales de comunicación para dicha acción

* **Tópico 5**: Este tema va encaminado cuando ya se posee solución a una solicitud o duda que hayan podido tener los usuarios. Es muy contundente con palabras como "evidencia" (el resumen de ejemplo brinda un buen acercamiento a la idea central del tópico "Ha sido atendida por mensaje interno y nos encontramos atentos para ayudar en caso de alguna inquietud adicional")

* **Tópico 6**: Bastante inclinado a brindar acompañamiento y generar cercania con el cliente, ya sea por una duda o una solicitud. Esta más relacionado propiamente a esa disposición por parte de la compañia al servicio amable.

* **Tópico 7**: Se solicita remisión de un caso como mensaje privado y detallar dicho proceso.

* **Tópico 8**: Solicitud de datos personales (Tal como se ve en el resumen "por favor envíenos su nombre y número de documento, únicamente por mensaje privado")

* **Tópico 9**: Claramente inclinado a brindar disculpas por fallas a nivel general.

* **Tópico 10**: Disculpas por inconvenientes presentados 
 

**SIMPLIFICACIÓN DE TÓPICOS**

Según mi criterio propio y habiendo interpretado cada uno de los tópicos puedo sugerir los siguientes temas en los cuales agrupar cada conjunto.

* **Respuesta general o genérica de Davivienda** (Tópicos 0 y 4)

* **Validación en inconvenientes** (Tópicos 1 y 3)

* **Solicitud de detalles del caso en mensaje privado** (Tópicos 2 y 7)

* **Ofreciendo disculpas** (Tópico 9 y 10)

* **Solución hallada o evidencia encontrada sobre algún caso en particular** (Tópico 5)

* **Disposición de servicio y acompañamiento al cliente ante cualquier problema o inquiertud** (Tópico 6)

* **Solicitud de datos personales por mensaje interno** (Tópico 8)